In [2]:
import os, sys
import pandas as pd

col_names = ['task', 'dataset', 'task2', 'eval_type', 'data_setting', 'shots', 'seed', 'f1-micro', 'f1-macro', 'accuracy']
runs = []

RESULTS_PATH = 'Students/' # Experts, LLM
folder_names = [folder for folder in os.listdir(os.path.join(RESULTS_PATH)) if os.path.isdir(os.path.join(RESULTS_PATH, folder)) and folder != '.ipynb_checkpoints']

for folder_name in folder_names:
    try:
        cond_parameters = folder_name.split('_')
        if cond_parameters[0] == 'acd':
            df = pd.read_csv(os.path.join(RESULTS_PATH, folder_name, 'metrics_asp.tsv'), sep = '\t')
        elif cond_parameters[0] == 'tasd':
            df = pd.read_csv(os.path.join(RESULTS_PATH, folder_name, 'result_asp_pol_phrases.tsv'), sep = '\t')
        else:
            df = pd.read_csv(os.path.join(RESULTS_PATH, folder_name, 'result_asp_pol.tsv'), sep = '\t')
                    
        df = df.set_index(df.columns[0])

        cond_parameters.append(df.loc['Micro-AVG', 'F1'])
        cond_parameters.append(df.loc['Macro-AVG', 'F1'])
        cond_parameters.append(df.loc['Micro-AVG', 'Accuracy'])
        runs.append(cond_parameters)
    except:
        pass

results_all = pd.DataFrame(runs, columns = col_names)
results_all["f1-micro"] = pd.to_numeric(results_all["f1-micro"], errors="coerce")


config_cols = ["dataset", "data_setting", "eval_type", "learning_rate", "epoch"]

# 2. Mittelung von 'orig-o' + 'orig-d' für jeden Seed + weitere Konfigs
# -> z. B. ["dataset", "task", "eval_type", "seed", "learning_rate", "epoch"]
group_cols_for_merge = [col for col in config_cols if col != 'data_setting'] + ['task', 'batch_size']
if 'seed' in results_all.columns:
    group_cols_for_merge.append('seed')
print(RESULTS_PATH)
results_all

Students/


,task,dataset,task2,eval_type,data_setting,shots,seed,f1-micro,f1-macro,accuracy
0,acsa,studentredo,acsa,test,orig-0,50,10,0.8643,0.8444,0.7611
1,acsa,studentredo,acsa,test,orig-0,50,15,0.8677,0.8506,0.7664
2,acsa,studentredo,acsa,test,orig-0,50,20,0.8670,0.8464,0.7652
3,acsa,studentredo,acsa,test,orig-0,50,25,0.8609,0.8422,0.7558
4,acsa,studentredo,acsa,test,orig-0,50,5,0.8616,0.8412,0.7568
5,tasd,students,tasd,test,orig-0,50,10,0.6107,0.5749,0.4395
6,tasd,students,tasd,test,orig-0,50,15,0.6217,0.5931,0.4510
7,tasd,students,tasd,test,orig-0,50,20,0.6315,0.5993,0.4615
8,tasd,students,tasd,test,orig-0,50,25,0.6379,0.6094,0.4683
9,tasd,students,tasd,test,orig-0,50,5,0.6122,0.5763,0.4412


In [3]:
import numpy as np

task = 'acsa'
data_setting = 'orig-0'
eval_count = 5
eval_type = 'test'

config_cols = ["data_setting", "eval_type", "shots"]

# Filtere nach den relevanten Parametern
df_filtered = results_all[np.logical_and.reduce([
    results_all['task'] == task, 
    results_all['data_setting'] == data_setting,
    results_all['eval_type'] == eval_type
])]

# Gruppiere und filtere auf Gruppen mit genau eval_count Einträgen
df_grouped = df_filtered.groupby(config_cols).filter(lambda x: len(x) == eval_count)

# Berechne dann den Durchschnitt nur über diese Gruppen
df_best_per_lang = df_grouped.groupby(config_cols)[["f1-micro", "f1-macro", "accuracy"]].mean().reset_index()

df_best_per_lang['f1-micro'] = df_best_per_lang['f1-micro'].apply(lambda x: round(x*100,2))
df_best_per_lang['f1-macro'] = df_best_per_lang['f1-macro'].apply(lambda x: round(x*100,2))
print(RESULTS_PATH)
print("Method: LLM-FS")
df_best_per_lang

Students/
Method: LLM-FS


,data_setting,eval_type,shots,f1-micro,f1-macro,accuracy
0,orig-0,test,50,86.43,84.5,0.76106
